In [23]:
!pwd

/Users/poderozita/z2022_1/REPOSITORY/NVCC


In [17]:
%%writefile smallest.cu 
#include <stdio.h>

__global__
void cuda_hello()
{
  printf(">> [smallest] Hello from GPU!\n");
}

int main()
{
    printf(">> [smallest] Hello from CPU!\n");   
    cuda_hello<<<1,1>>>();
    return 0;
}

Overwriting smallest.cu


In [22]:
%%writefile CMakeLists.txt 

cmake_minimum_required(VERSION 3.8 FATAL_ERROR)
project(hello LANGUAGES CXX CUDA)
enable_language(CUDA)
add_executable(smallest.x ../smallest.cu)

enable_testing()
add_test(NAME smallest COMMAND smallest.x) 

Overwriting CMakeLists.txt


In [1]:
%%writefile Dockerfile.ci 

FROM ubuntu:focal 
RUN apt-get --yes -qq update 
RUN DEBIAN_FRONTEND=noninteractive TZ=Etc/UTC apt-get -y install tzdata
RUN apt-get --yes -qq update 

RUN apt-get --yes -qq install wget xz-utils
RUN apt-get --yes -qq install build-essential  
RUN apt-get --yes -qq install gcc g++  
RUN apt-get --yes -qq install cmake 
RUN apt-get --yes -qq install cmake-curses-gui ## ccmake 
RUN apt-get --yes -qq clean
RUN rm -rf /var/lib/apt/lists/*


Overwriting Dockerfile.ci


In [21]:
%%writefile GET_NVCC.sh 

## https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html
SETUP_NVCC()
{ 
  echo "wget..."
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_nvcc/linux-x86_64/cuda_nvcc-linux-x86_64-${RELEASE}-archive.tar.xz
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_cudart/linux-x86_64/cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
  
  echo "tar..."
  rm -rf cuda_nvcc-linux-x86_64-${RELEASE}-archive
  tar -xf cuda_nvcc-linux-x86_64-${RELEASE}-archive.tar.xz #> /dev/null 
  #ls -la cuda_nvcc-linux-x86_64-${RELEASE}-archive
}


SETUP_CUDART()
{ 
  echo "wget..."
  wget https://developer.download.nvidia.com/compute/cuda/redist/cuda_cudart/linux-x86_64/cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
  
  echo "tar..."
  rm -rf cuda_cudart-linux-x86_64-${RELEASE}-archive
  tar -xf cuda_cudart-linux-x86_64-${RELEASE}-archive.tar.xz
  #ls -la cuda_cudart-linux-x86_64-${RELEASE}-archive
}


INIT()
{
  g++ --version   ## g++ (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
  cmake --version ## cmake version 3.16.3

  NVCC_PATH=/home/cuda_nvcc-linux-x86_64-${RELEASE}-archive
  CUDART_PATH=/home/cuda_cudart-linux-x86_64-${RELEASE}-archive

  NVCC=$NVCC_PATH/bin/nvcc
  $NVCC --version

  rm smallest.x
  $NVCC -o smallest.x ../smallest.cu -I$CUDART_PATH/include -L$CUDART_PATH/lib

  export CPLUS_INCLUDE_PATH=$CPLUS_INCLUDE_PATH:$CUDART_PATH/include 
  export LIBRARY_PATH=$LIBRARY_PATH:$CUDART_PATH/lib 
  export CUDACXX=$NVCC 

  rm smallest.x  
  $CUDACXX  -o smallest.x ../smallest.cu  
  ./smallest.x 

  rm smallest.x  
  rm -rf CMakeCache.txt CMakeFiles  
  cmake ..
  make
  ctest
}


RELEASE=11.5.50 
#SETUP_NVCC
#SETUP_CUDART
INIT




Overwriting GET_NVCC.sh


In [3]:
%%writefile workflow.sh 


DOCKER_BUILD()
{
  docker build . \
    --file Dockerfile.ci \
    --tag $IMAGE_NAME  \
    > /dev/null 
}


DOCKER_IMAGES()
{
  IMAGE_ID=$(docker images --filter=reference=$IMAGE_NAME --format "{{.ID}}") 
  #echo "IMAGE_ID:", $IMAGE_ID    
}


DOCKER_RUN()
{
  docker run \
    --rm \
    --name ${CONTAINER_NAME} \
    --volume ${PWD}:/home \
    --workdir /home \
    -i ${IMAGE_NAME} \
    ${TOBEEXECUTED}
}


CONTAINER_NAME=nvcc_container
IMAGE_NAME=nvcc_image 

DOCKER_BUILD 

TOBEEXECUTED="ls -la"
DOCKER_RUN

TOBEEXECUTED="pwd"
DOCKER_RUN

TOBEEXECUTED="bash GET_NVCC.sh"
#DOCKER_RUN

#
# docker run --rm --name nvcc_container --volume ${PWD}:/home --workdir /home -i nvcc_image bash GET_NVCC.sh
# 

Overwriting workflow.sh


In [4]:
%%bash 

bash workflow.sh  


#1 [internal] load build definition from Dockerfile.ci
#1 sha256:28ddd31595063bc31cca165f0c661d4e4eee2f7e24c901cc4e15bb21dd89fedd
#1 transferring dockerfile: 439B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:5433478f5e5b8d837fd5e08aaf224a4ccefe00bc18c8c99e213dd5d480468d04
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:focal
#3 sha256:d8099b9e2a8d5949e23c58d0d8ac7c3b4e8785ea131ce18291d94cefef37ed4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:4c151b2f0e2cf951f118ad101feae14b31531e9edf35d1826722f4d736296b71
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:focal
#3 sha256:d8099b9e2a8d5949e23c58d0d8ac7c3b4e8785ea131ce18291d94cefef37ed4b
#3 DONE 1.9s

#5 [1/9] FROM docker.io/library/ubuntu:focal@sha256:47f14534bda344d9fe6ffd6effb95eefe579f4be0d508b7445cf77f61a0e5724
#5 sha256:f972fadbf0cc603a95018407b982cb0d43028f695be30ed267ddd9da82779cb8
#5 DONE 0.0s

#6 [

#10 12.70 Selecting previously unselected package liblsan0:amd64.
#10 12.70 Preparing to unpack .../32-liblsan0_10.3.0-1ubuntu1~20.04_amd64.deb ...
#10 12.70 Unpacking liblsan0:amd64 (10.3.0-1ubuntu1~20.04) ...
#10 12.84 Selecting previously unselected package libtsan0:amd64.
#10 12.84 Preparing to unpack .../33-libtsan0_10.3.0-1ubuntu1~20.04_amd64.deb ...
#10 12.84 Unpacking libtsan0:amd64 (10.3.0-1ubuntu1~20.04) ...
#10 13.20 Selecting previously unselected package libubsan1:amd64.
#10 13.21 Preparing to unpack .../34-libubsan1_10.3.0-1ubuntu1~20.04_amd64.deb ...
#10 13.21 Unpacking libubsan1:amd64 (10.3.0-1ubuntu1~20.04) ...
#10 13.35 Selecting previously unselected package libquadmath0:amd64.
#10 13.35 Preparing to unpack .../35-libquadmath0_10.3.0-1ubuntu1~20.04_amd64.deb ...
#10 13.35 Unpacking libquadmath0:amd64 (10.3.0-1ubuntu1~20.04) ...
#10 13.40 Selecting previously unselected package libgcc-9-dev:amd64.
#10 13.40 Preparing to unpack .../36-libgcc-9-dev_9.4.0-1ubuntu1~20.04.

#10 18.93 Unpacking gpg-agent (2.2.19-3ubuntu2.1) ...
#10 19.00 Selecting previously unselected package gpg-wks-client.
#10 19.00 Preparing to unpack .../72-gpg-wks-client_2.2.19-3ubuntu2.1_amd64.deb ...
#10 19.00 Unpacking gpg-wks-client (2.2.19-3ubuntu2.1) ...
#10 19.06 Selecting previously unselected package gpg-wks-server.
#10 19.06 Preparing to unpack .../73-gpg-wks-server_2.2.19-3ubuntu2.1_amd64.deb ...
#10 19.06 Unpacking gpg-wks-server (2.2.19-3ubuntu2.1) ...
#10 19.23 Selecting previously unselected package gpgsm.
#10 19.23 Preparing to unpack .../74-gpgsm_2.2.19-3ubuntu2.1_amd64.deb ...
#10 19.25 Unpacking gpgsm (2.2.19-3ubuntu2.1) ...
#10 19.39 Selecting previously unselected package gnupg.
#10 19.40 Preparing to unpack .../75-gnupg_2.2.19-3ubuntu2.1_all.deb ...
#10 19.49 Unpacking gnupg (2.2.19-3ubuntu2.1) ...
#10 19.62 Selecting previously unselected package libalgorithm-diff-perl.
#10 19.62 Preparing to unpack .../76-libalgorithm-diff-perl_1.19.03-2_all.deb ...
#10 19.63 

#10 22.75 update-alternatives: using /usr/bin/g++ to provide /usr/bin/c++ (c++) in auto mode
#10 22.76 update-alternatives: warning: skip creation of /usr/share/man/man1/c++.1.gz because associated file /usr/share/man/man1/g++.1.gz (of link group c++) doesn't exist
#10 22.76 Setting up build-essential (12.8ubuntu1.1) ...
#10 22.77 Setting up libalgorithm-diff-xs-perl (0.04-6) ...
#10 22.78 Setting up libalgorithm-merge-perl (0.08-3) ...
#10 22.79 Setting up libldap-2.4-2:amd64 (2.4.49+dfsg-2ubuntu1.8) ...
#10 22.81 Setting up dirmngr (2.2.19-3ubuntu2.1) ...
#10 23.12 Setting up gpg-wks-client (2.2.19-3ubuntu2.1) ...
#10 23.12 Setting up gnupg (2.2.19-3ubuntu2.1) ...
#10 23.13 Processing triggers for libc-bin (2.31-0ubuntu9.7) ...
#10 DONE 23.5s

#11 [7/9] RUN apt-get --yes -qq install cmake
#11 sha256:5419f072073a83cea0f4d6fda8a9e38032ad5ff500ef31bf76d88e93a1a929d7
#11 3.958 debconf: delaying package configuration, since apt-utils is not installed
#11 4.031 Selecting previously unselec

total 35204
drwxr-xr-x 10 root  root      320 May  5 06:50 .
drwxr-xr-x  1 root  root     4096 May  5 06:50 ..
-rw-r--r--  1 root  root     6148 May  5 04:41 .DS_Store
drwxr-xr-x  4 root  root      128 May  5 06:50 .ipynb_checkpoints
-rw-r--r--  1 root  root      392 May  5 06:50 Dockerfile.ci
-rw-r--r--  1 root  root      546 May  5 06:50 GET_NVCC.sh
drwxr-xr-x  7 25503 tape      224 Aug 25  2021 cuda_nvcc-linux-x86_64-11.4.120-archive
-rw-r--r--  1 root  root 35913232 Aug 30  2021 cuda_nvcc-linux-x86_64-11.4.120-archive.tar.xz
-rw-r--r--  1 root  root    41090 May  5 06:50 nvcc01a.ipynb
-rw-r--r--  1 root  root      595 May  5 06:50 workflow.sh
/home
